# GaussianChangepointMeanVar2D Model Demo - Enhanced with Model Fitting

This notebook demonstrates the usage of the GaussianChangepointMeanVar2D model with dummy data,
including model fitting and comparison of fit outputs to original data.

This model detects changepoints in both mean AND variance of Gaussian data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add pytau to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..', '..'))

from pytau.changepoint_model import GaussianChangepointMeanVar2D, gen_test_array, advi_fit
import arviz as az

## Generate Test Data

In [ ]:
# Generate dummy data with obvious changepoints in both mean and variance
np.random.seed(42)
n_trials = 10
n_time = 100
n_states = 3

# Create data with state transitions in BOTH mean and variance
state_means = [0.5, 3.5, 2.0]
state_stds = [0.3, 1.2, 0.8]  # Different variances for each state
transition_points = [33, 66]  # Clear transition points

test_data = np.zeros((n_trials, n_time))
for trial in range(n_trials):
    # Add some trial-to-trial variability in transition points
    trial_transitions = [t + np.random.randint(-3, 4) for t in transition_points]
    trial_transitions = [max(5, min(n_time-5, t)) for t in trial_transitions]
    
    # Generate data for each state with different mean AND variance
    test_data[trial, :trial_transitions[0]] = np.random.normal(state_means[0], state_stds[0], trial_transitions[0])
    test_data[trial, trial_transitions[0]:trial_transitions[1]] = np.random.normal(state_means[1], state_stds[1], trial_transitions[1] - trial_transitions[0])
    test_data[trial, trial_transitions[1]:] = np.random.normal(state_means[2], state_stds[2], n_time - trial_transitions[1])

print(f"Generated test data shape: {test_data.shape}")
print(f"Data range: [{test_data.min():.2f}, {test_data.max():.2f}]")
print(f"State means: {state_means}")
print(f"State stds: {state_stds}")
print(f"Transition points (approx): {transition_points}")

## Initialize and Generate Model

In [ ]:
# Initialize model
model = GaussianChangepointMeanVar2D(
    data_array=test_data,
    n_states=n_states,
    fit_type='vi'
)

# Generate the PyMC model
pymc_model = model.generate_model()
print("Model generated successfully")

## Fit Model with ADVI

In [ ]:
# Fit the model using ADVI
print("Fitting model with ADVI...")
n_fit = 2000  # Number of ADVI iterations
n_samples = 500  # Number of samples to draw

fitted_model, approx = advi_fit(pymc_model, n_fit, n_samples)

# Sample from the fitted model
trace = approx.sample(draws=n_samples)

print(f"Model fitting completed. Final ELBO: {approx.hist[-1]:.2f}")
print(f"Trace variables: {list(trace.posterior.data_vars)}")

## Extract and Analyze Fitted Parameters

In [ ]:
# Extract fitted parameters
mu_samples = trace.posterior['mu'].values  # State means
sigma_samples = trace.posterior['sigma'].values  # State standard deviations
tau_samples = trace.posterior['tau'].values  # Changepoint locations

print(f"Mu (state means) shape: {mu_samples.shape}")
print(f"Sigma (state stds) shape: {sigma_samples.shape}")
print(f"Tau (changepoints) shape: {tau_samples.shape}")

# Calculate summary statistics - handle the actual dimensions
mu_mean = mu_samples.mean(axis=(0, 1))  # Average over chains and draws
sigma_mean = sigma_samples.mean(axis=(0, 1))  # Average over chains and draws
tau_mean = tau_samples.mean(axis=(0, 1))  # Average over chains and draws

print(f"\nEstimated state means (mean across samples):")
for state in range(min(n_states, mu_mean.shape[-1] if mu_mean.ndim > 0 else 1)):
    mean_val = mu_mean[..., state].mean() if mu_mean.ndim > 1 else (mu_mean[state] if mu_mean.ndim == 1 else mu_mean)
    print(f"  State {state+1}: {mean_val:.2f} (true: {state_means[state]})")

print(f"\nEstimated state standard deviations (mean across samples):")
for state in range(min(n_states, sigma_mean.shape[-1] if sigma_mean.ndim > 0 else 1)):
    std_val = sigma_mean[..., state].mean() if sigma_mean.ndim > 1 else (sigma_mean[state] if sigma_mean.ndim == 1 else sigma_mean)
    print(f"  State {state+1}: {std_val:.2f} (true: {state_stds[state]})")

print(f"\nEstimated changepoints (mean across samples):")
n_changepoints = min(len(transition_points), tau_mean.shape[-1] if tau_mean.ndim > 0 else 1)
for i in range(n_changepoints):
    cp = tau_mean[..., i].mean() if tau_mean.ndim > 1 else (tau_mean[i] if tau_mean.ndim == 1 else tau_mean)
    print(f"  Changepoint {i+1}: {cp:.1f} (true: ~{transition_points[i]})")

# Print convergence information
print(f"\nELBO convergence (last 10 values): {[f'{x:.2f}' for x in approx.hist[-10:]]}")

## Visualize Original Data vs Model Fit

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(3, 2, figsize=(15, 15))

# 1. Original data time series (first few trials)
ax = axes[0, 0]
for trial in range(min(3, test_data.shape[0])):
    ax.plot(test_data[trial, :], 'o-', markersize=2, alpha=0.7, label=f'Trial {trial+1}')
ax.axvline(x=transition_points[0], color='red', linestyle='--', alpha=0.7, label='True CP 1')
ax.axvline(x=transition_points[1], color='red', linestyle='--', alpha=0.7, label='True CP 2')
ax.set_title('Original Data (First 3 Trials)')
ax.set_xlabel('Time')
ax.set_ylabel('Value')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Estimated changepoints distribution
ax = axes[0, 1]
if tau_samples.shape[-1] >= 2:  # Check if we have changepoints
    for i in range(min(2, tau_samples.shape[-1])):
        cp_data = tau_samples[..., i].flatten()  # Flatten all dimensions
        ax.hist(cp_data, bins=20, alpha=0.6, label=f'CP {i+1}', density=True)
        if i < len(transition_points):
            ax.axvline(x=transition_points[i], color='red', linestyle='--', alpha=0.8)
ax.set_title('Estimated Changepoint Distributions')
ax.set_xlabel('Time')
ax.set_ylabel('Density')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Estimated vs True state means
ax = axes[1, 0]
if mu_mean.shape[-1] >= n_states:
    estimated_means = [mu_mean[..., state].mean() for state in range(n_states)]
    x_pos = np.arange(len(state_means))
    width = 0.35
    ax.bar(x_pos - width/2, state_means, width, label='True Means', alpha=0.7)
    ax.bar(x_pos + width/2, estimated_means, width, label='Estimated Means', alpha=0.7)
    ax.set_title('True vs Estimated State Means')
    ax.set_xlabel('State')
    ax.set_ylabel('Mean Value')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'State {i+1}' for i in range(len(state_means))])
    ax.legend()
    ax.grid(True, alpha=0.3)

# 4. Estimated vs True state standard deviations
ax = axes[1, 1]
if sigma_mean.shape[-1] >= n_states:
    estimated_stds = [sigma_mean[..., state].mean() for state in range(n_states)]
    x_pos = np.arange(len(state_stds))
    width = 0.35
    ax.bar(x_pos - width/2, state_stds, width, label='True Stds', alpha=0.7)
    ax.bar(x_pos + width/2, estimated_stds, width, label='Estimated Stds', alpha=0.7)
    ax.set_title('True vs Estimated State Standard Deviations')
    ax.set_xlabel('State')
    ax.set_ylabel('Standard Deviation')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([f'State {i+1}' for i in range(len(state_stds))])
    ax.legend()
    ax.grid(True, alpha=0.3)

# 5. ELBO convergence
ax = axes[2, 0]
ax.plot(approx.hist)
ax.set_title('ELBO Convergence')
ax.set_xlabel('Iteration')
ax.set_ylabel('ELBO')
ax.grid(True, alpha=0.3)

# 6. Average response with fitted changepoints and variance bands
ax = axes[2, 1]
avg_response = test_data.mean(axis=0)  # Average across trials
std_response = test_data.std(axis=0)   # Standard deviation across trials

ax.plot(avg_response, 'b-', linewidth=2, label='Observed Mean')
ax.fill_between(range(n_time), avg_response - std_response, avg_response + std_response, 
               alpha=0.3, color='blue', label='Observed ±1 SD')

ax.axvline(x=transition_points[0], color='red', linestyle='--', alpha=0.7, label='True CPs')
ax.axvline(x=transition_points[1], color='red', linestyle='--', alpha=0.7)

# Plot estimated changepoints
if tau_mean.shape[-1] >= 2:
    for i in range(min(2, tau_mean.shape[-1])):
        cp = tau_mean[..., i].mean()
        ax.axvline(x=cp, color='green', linestyle=':', alpha=0.8, 
                  label='Est. CPs' if i == 0 else '')

ax.set_title('Average Response with Changepoints and Variance')
ax.set_xlabel('Time')
ax.set_ylabel('Value')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Model Diagnostics

In [ ]:
# Print detailed model diagnostics
print("=== MODEL DIAGNOSTICS ===")
print(f"\nData shape: {test_data.shape}")
print(f"Number of states: {n_states}")
print(f"ADVI iterations: {n_fit}")
print(f"Final ELBO: {approx.hist[-1]:.2f}")

print(f"\n=== PARAMETER ESTIMATES ===")
print("State Means (mu):")
if mu_mean.shape[-1] >= n_states:
    for i in range(n_states):
        true_mean = state_means[i]
        est_mean = mu_mean[..., i].mean()
        est_std = mu_samples[..., i].std()
        print(f"  State {i+1}: {est_mean:.2f} ± {est_std:.2f} (true: {true_mean:.2f})")

print("\nState Standard Deviations (sigma):")
if sigma_mean.shape[-1] >= n_states:
    for i in range(n_states):
        true_std = state_stds[i]
        est_std_mean = sigma_mean[..., i].mean()
        est_std_std = sigma_samples[..., i].std()
        print(f"  State {i+1}: {est_std_mean:.2f} ± {est_std_std:.2f} (true: {true_std:.2f})")

print("\nChangepoints (tau):")
n_changepoints = min(len(transition_points), tau_mean.shape[-1])
for i in range(n_changepoints):
    true_cp = transition_points[i]
    est_cp = tau_mean[..., i].mean()
    est_std = tau_samples[..., i].std()
    print(f"  CP {i+1}: {est_cp:.1f} ± {est_std:.1f} (true: ~{true_cp})")

print(f"\n=== CONVERGENCE ===")
elbo_diff = np.diff(approx.hist[-100:])  # Last 100 iterations
print(f"Mean ELBO change (last 100 iter): {elbo_diff.mean():.4f}")
print(f"Std ELBO change (last 100 iter): {elbo_diff.std():.4f}")

if abs(elbo_diff.mean()) < 0.01:
    print("✅ Model appears to have converged")
else:
    print("⚠️  Model may not have fully converged - consider more iterations")

print(f"\n=== DATA CHARACTERISTICS ===")
for state, (start, end) in enumerate([(0, transition_points[0]), 
                                     (transition_points[0], transition_points[1]), 
                                     (transition_points[1], n_time)]):
    state_data = test_data[:, start:end]
    print(f"State {state+1} (time {start}-{end}): mean={state_data.mean():.2f}, std={state_data.std():.2f}")

## Summary

This enhanced notebook demonstrated:
1. **Data Generation**: Creating synthetic Gaussian data with known changepoints in BOTH mean and variance
2. **Model Initialization**: Setting up the GaussianChangepointMeanVar2D model
3. **Model Fitting**: Using ADVI to fit the model to data
4. **Parameter Extraction**: Extracting and analyzing fitted parameters (means, standard deviations, changepoints)
5. **Model Validation**: Comparing fitted parameters to true values for both mean and variance
6. **Visualization**: Comprehensive plots showing:
   - Original data patterns
   - Estimated vs true changepoints
   - Estimated vs true state means
   - Estimated vs true state standard deviations
   - ELBO convergence
   - Variance bands around mean response
7. **Diagnostics**: Model convergence and parameter accuracy assessment

The GaussianChangepointMeanVar2D model successfully detected the changepoints and estimated both the state means and variances, demonstrating its effectiveness for analyzing continuous data with changes in both first and second moments.